In [1]:
import requests
import pandas as pd
import os
 
# Henter data fra API
kompispassord = os.environ["kompispassord"]
kompisbrukar = os.environ["kompisbrukar"]
liste = [10013727, 10013736]
 
temp = []
for org_id in liste:
    url = f"https://hvl-dev.outsystemsenterprise.com/CV_API/rest/v1/HentAnsatte?Orgenhet={org_id}"
    resultat = requests.get(url, auth=(kompisbrukar, kompispassord))
    temp.append(pd.DataFrame(resultat.json()))
 
# Kombinerer alle data i én DataFrame
df = pd.concat(df for df in temp if not df.empty).reset_index(drop=True)
 
# Funksjon for å eksplodere JSON-lister og mappe om til riktig format
def explode_and_rename(df, column, rename_map):
    """ Eksploderer en JSON-kolonne og mapper om feltene """
    if column in df.columns:
        df[column] = df[column].apply(lambda x: eval(x) if isinstance(x, str) else x)  # Konverter JSON-streng til liste
        df_exp = df.explode(column).reset_index(drop=True)  # Lager en rad per element i listen
        df_exp = pd.concat([df_exp.drop(columns=[column]), df_exp[column].apply(pd.Series)], axis=1)  # Ekspanderer JSON-objekter
        return df_exp.rename(columns=rename_map)  # Omdøper kolonner til ønskede navn
    else:
        print(f"Advarsel: Kolonnen '{column}' finnes ikke i datasettet.")
        return df  # Returnerer df uendret
 
# Definerer kolonnenavnene for de forskjellige eksplosjonene
prosjekt_map = {
    "tittel": "Prosjekt_tittel",
    "rolle": "Prosjekt_rolle",
    "eier": "Prosjekt_eier",
    "fradato": "Prosjekt_fradato",
    "tildato": "Prosjekt_tildato",
    "totalbudsjett": "Prosjekt_totalbudsjett"
}
 
publikasjon_map = {
    "kategori": "Publikasjon_kategori",
    "tittel": "Publikasjon_tittel",
    "publiseringsaar": "Publikasjon_publiseringsår",
    "kanal": "Publikasjon_kanal",
    "forfatter": "Publikasjon_forfatter"
}
 
utmerkelse_map = {
    "tittel": "Utmerkelse_tittel",
    "aar": "Utmerkelse_år",
    "utsteder": "Utmerkelse_utsteder",
    "beskrivelse": "Utmerkelse_beskrivelse"
}
 
# Eksploderer JSON-kolonnene og mapper om kolonnenavnene
df_prosjekt = explode_and_rename(df, "prosjekt", prosjekt_map)
df_publikasjon = explode_and_rename(df, "publikasjon", publikasjon_map)
df_utmerkelse = explode_and_rename(df, "utmerkelse", utmerkelse_map)
 
# Setter NaN til tom streng for lesbarhet
df_prosjekt.fillna("", inplace=True)
df_publikasjon.fillna("", inplace=True)
df_utmerkelse.fillna("", inplace=True)
 
# Kombinerer de eksploderte datasettene riktig
df_final = pd.concat([df_prosjekt, df_publikasjon, df_utmerkelse]).reset_index(drop=True)
 
# Lagrer til CSV for import i Power BI
df_final.to_csv("output.csv", index=False, sep=";")
 
print("Filen 'output.csv' er klar!")

Filen 'output.csv' er klar!


C:\Users\AKV\AppData\Local\Temp\ipykernel_23328\357937755.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_prosjekt.fillna("", inplace=True)
C:\Users\AKV\AppData\Local\Temp\ipykernel_23328\357937755.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_publikasjon.fillna("", inplace=True)
C:\Users\AKV\AppData\Local\Temp\ipykernel_23328\357937755.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_utmerkelse.fillna("", inplace=True)
